In [27]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity 
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
import spacy
import gensim.downloader as api
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models import KeyedVectors
import json
import gensim.downloader as api


In [28]:
from utils import * 
from datasets import load_dataset
import json
import gensim.downloader as api


In [63]:


info = api.info()
print(json.dumps(info, indent=4))

# Retry downloading the pre-trained Word2Vec model
pretrained_model_path = api.load('word2vec-google-news-300', return_path=True)

{
    "corpora": {
        "semeval-2016-2017-task3-subtaskBC": {
            "num_records": -1,
            "record_format": "dict",
            "file_size": 6344358,
            "reader_code": "https://github.com/RaRe-Technologies/gensim-data/releases/download/semeval-2016-2017-task3-subtaskB-eng/__init__.py",
            "license": "All files released for the task are free for general research use",
            "fields": {
                "2016-train": [
                    "..."
                ],
                "2016-dev": [
                    "..."
                ],
                "2017-test": [
                    "..."
                ],
                "2016-test": [
                    "..."
                ]
            },
            "description": "SemEval 2016 / 2017 Task 3 Subtask B and C datasets contain train+development (317 original questions, 3,169 related questions, and 31,690 comments), and test datasets in English. The description of the tasks and the collect

In [61]:

# Load MultiWOZ 2.2 dataset
dataset = load_dataset("multi_woz_v22")

# Print available splits (train, validation, test)
print(dataset)

# View a sample dialogue
print(dataset["train"][0])

DatasetDict({
    train: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 8437
    })
    validation: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['dialogue_id', 'services', 'turns'],
        num_rows: 1000
    })
})
{'dialogue_id': 'PMUL4398.json', 'services': ['restaurant', 'hotel'], 'turns': {'turn_id': ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9', '10', '11'], 'speaker': [0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1], 'utterance': ['i need a place to dine in the center thats expensive', 'I have several options for you; do you prefer African, Asian, or British food?', 'Any sort of food would be fine, as long as it is a bit expensive. Could I get the phone number for your recommendation?', 'There is an Afrian place named Bedouin in the centre. How does that sound?', 'Sounds good, could I get that phone number? Also, could you recommend me an expensive hotel?', "Bedo

In [17]:
total_categories = list(set(element for sublist in dataset['train']['services'] for element in sublist))

print(total_categories)


# Save to JSON file
with open("data/categories.json", "w") as json_file:
    json.dump(total_categories, json_file)



['hotel', 'train', 'taxi', 'bus', 'hospital', 'restaurant', 'attraction']


In [64]:

nlp = spacy.load("en_core_web_sm")
# Load the downloaded model
pretrained_model = KeyedVectors.load_word2vec_format(pretrained_model_path, binary=True)


In [65]:
def cosine_similarity(vec1, vec2):
    return np.dot(vec1, vec2) / (np.linalg.norm(vec1) * np.linalg.norm(vec2))

In [20]:
def calculate_similarity_indices(vector1, vector2):
    # Calculate the cosine similarity between each row in vector1 and vector2
    similarities = cosine_similarity(vector1, vector2)

    # For each row in vector1, find the index of the most similar row in vector2 
    most_similar_indices = np.argmax(similarities, axis=1)
    
    return most_similar_indices


def tokenize_sentence(sentence):
    
    doc = nlp(sentence)
    tokens = [token.text.lower() for token in doc if not token.is_punct]
    return tokens


def embed_sentence(sentence):
    # Split the sentence into tokens
    tokens = tokenize_sentence(sentence)
    
    # Initialize an empty array to store the word embeddings
    embeddings = np.zeros(pretrained_model.vector_size)
    
    n = 0
    # Iterate over each token in the sentence
    for token in tokens:
        # Check if the token is present in the pretrained word2vec model
        if token in pretrained_model:
            # Add the word embedding to the sentence embeddings
            embeddings += pretrained_model[token]
            n+=1
    
    # Normalize the sentence embeddings
    embeddings /= n+1
    
    return np.array(embeddings)

In [6]:
class ChatBot:

    def __init__(self):
        
        self.dataframes = {}
        self.themes = total_categories
        self.name = 'MarioBot'

        self.presentation = 'Hi, my name is MarioBot, I am a chatbot. I am here to help you with any questions you may have regarding: '


        for theme in self.themes:

            temp_df = pd.read_csv(f"data/{theme}.csv")
            self.dataframes[theme] = temp_df

        self.nlp = None
        self.w2v = None

    def load_models(self):

        self.nlp = spacy.load("en_core_web_sm")

        pretrained_model_path = "/Users/rocco02/gensim-data/word2vec-google-news-300/word2vec-google-news-300.gz"
        self.w2v = KeyedVectors.load_word2vec_format(pretrained_model_path, binary=True)

    
    def get_dialogue(self):

        print(f"{self.name}: {self.presentation + ', '.join(self.themes)}.\nPlease ask me a question:")

        self.load_models()
        
        while True:

            query = input("Insert the question: ")

            print(f"\nUser: {query}")

            matching_words =  self.find_matching_words(query)

            topic = matching_words[0]

            if len(matching_words) > 1:
                
                print(f'\n{self.name}: Looks like you are asking about multiple topics. We will solve one topic at time to avoid confusion. We start with the first topic: {matching_words[0]}.')

            elif len(matching_words) == 0:

                print(f"\n{self.name}: I am sorry, I do not have information about that topic. Try to rephrase the question or ask me something else.")

            
            answer = self.find_best_answer(query, topic)
            print(f"\n{self.name}: {answer}")

        
    def tokenize_sentence(self, sentence):
        
        doc = self.nlp(sentence)
        tokens = [token.text.lower() for token in doc if not token.is_punct]
        return tokens


    def embed_sentence(self, sentence):
        # Split the sentence into tokens
        tokens = tokenize_sentence(sentence)
        
        # Initialize an empty array to store the word embeddings
        embeddings = np.zeros(self.w2v.vector_size)
        
        n = 0
        # Iterate over each token in the sentence
        for token in tokens:
            # Check if the token is present in the pretrained word2vec model
            if token in self.w2v:
                # Add the word embedding to the sentence embeddings
                embeddings += self.w2v[token]
                n+=1
        
        # Normalize the sentence embeddings
        embeddings /= n+1
        
        return embeddings


    def find_matching_words(self, query):
        return [word for word in self.themes if word in query]
    
    def find_best_answer(self, query, topic):
        
        embed_query = embed_sentence(query)
        df = self.dataframes[topic]['question_embeddings'].to_numpy()

        most_similar_responce = calculate_similarity_indices(embed_query, df)

        predicted_sentence = df.iloc[most_similar_responce[0]]['answer']

        return predicted_sentence
        

In [9]:
c = ChatBot()
c.dataframes['restaurant'].head()

,question,answer
0,i need a place to dine in the center thats exp...,I have several options for you; do you prefer ...
1,"Any sort of food would be fine, as long as it ...",There is an Afrian place named Bedouin in the ...
2,"Sounds good, could I get that phone number? Al...",Bedouin's phone is 01223367660. As far as hote...
3,Hi there! Can you give me some info on Cityroomz?,"Cityroomz is located at Sleeperz Hotel, Statio..."
4,Yes please. I need it for 7 people for 3 night...,How many days would you like to book it for?


## PRECOMPUTE THE DATAFRAMES

In [99]:
all_df = {}

for df_name in total_categories:
    all_df[df_name] = pd.read_csv(f"data/{df_name}.csv")

print(all_df)

{'hospital':                                               question  \
0                Could you find me a hospital in town?   
1    Do they have a transitional care department? I...   
2    I'm hurt and I need to find a hospital. Oh, an...   
3       Thank you. Can I have the postcode too please.   
4                  where is the Addenbrookes Hospital?   
..                                                 ...   
214                              I need the post code.   
215  I've been injured and I need a hospital near m...   
216  May I have the full hospital address and their...   
217  I'm looking for the Addenbrookes Hospital and ...   
218  Can you also tell me the postcode for the hosp...   

                                                answer  
0    Yes, Addenbrookes Hospital is in your area, wo...  
1    Yes they have transitional care. Their post co...  
2    Hello, I'm sorry you are injured. Addenbrookes...  
3                               The postcode is CB20QQ  
4    

In [71]:
embed_sentence('ciao a tutti').dtype

dtype('float64')

In [100]:
for df_name, df in all_df.items():
    # df['question_embeddings'] = df['question'].apply(embed_sentence)
    # df.to_csv(f"word2vec_data/{df_name}.csv", index=False)
    # df['question_embeddings'] = df['question'].apply(lambda x: " ".join(map(str, embed_sentence(x))))
    df.to_csv(f"word2vec_data/{df_name}.csv", index=False)
    np.save(f"word2vec_data/{df_name}_embeddings.npy", np.vstack(df['question'].apply(embed_sentence)))
    print(f"Saved {df_name}.csv")

Saved hospital.csv
Saved hotel.csv
Saved taxi.csv
Saved train.csv
Saved restaurant.csv
Saved bus.csv
Saved attraction.csv


In [78]:
for df_name in total_categories:

    a = pd.read_csv(f"word2vec_data/{df_name}.csv")
    print(a.head())

                                            question  \
0              Could you find me a hospital in town?   
1  Do they have a transitional care department? I...   
2  I'm hurt and I need to find a hospital. Oh, an...   
3     Thank you. Can I have the postcode too please.   
4                where is the Addenbrookes Hospital?   

                                              answer  \
0  Yes, Addenbrookes Hospital is in your area, wo...   
1  Yes they have transitional care. Their post co...   
2  Hello, I'm sorry you are injured. Addenbrookes...   
3                             The postcode is CB20QQ   
4  It's located at Hills Rd, Cambridge, you can r...   

                                 question_embeddings  
0  0.0694427490234375 0.03167724609375 0.03784179...  
1  -0.021839141845703125 0.022182464599609375 0.0...  
2  -0.04683467320033482 0.024152483258928572 0.03...  
3  -0.0166259765625 0.0154541015625 0.01789550781...  
4  0.000445556640625 0.017529296875 0.07338867187..

## Test chatbot

In [32]:
from chatbot import ChatBot


In [10]:
nlp = spacy.load("en_core_web_sm")

pretrained_model_path = api.load('word2vec-google-news-300', return_path=True)
w2v = KeyedVectors.load_word2vec_format(pretrained_model_path, binary=True)

In [41]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity 
from gensim.models import Word2Vec
from nltk.tokenize import word_tokenize
import numpy as np
import spacy
import gensim.downloader as api
from gensim.models import Word2Vec
from gensim.utils import simple_preprocess
from gensim.models import KeyedVectors
from utils import * 
import json
import gensim.downloader as api


class ChatBot:

    def __init__(self, nlp, w2v):
        
        self.dataframes = {}
        self.themes = None
        self.name = 'MarioBot'
        self.selected_theme = None
        self.theme_embeddings = []
        self.presentation = 'Hi, my name is MarioBot, I am a chatbot. I am here to help you with any questions you may have regarding: '

        self.current_embedding = None
        self.nlp = nlp
        self.w2v = w2v

        self.load_models()
        
    def load_models(self):

        with open("data/categories.json", "r") as json_file:
            self.themes = json.load(json_file)

        for theme in self.themes:

            temp_df = pd.read_csv(f"word2vec_data/{theme}.csv")
            self.dataframes[theme] = temp_df
            self.theme_embeddings.append(w2v[theme])
            
    def get_dialogue(self):

        print(f"{self.name}: {self.presentation + ', '.join(self.themes)}.\nPlease ask me a question:")
        
        while True:

            query = input("Insert the question: ")
            if query == '':
                break
            
            print(f"\nUser: {query}")

            # if self.selected_theme is None:
            #     matching_words =  self.find_matching_words(query)


            #     if len(matching_words) > 1:
            #         print(f'\n{self.name}: Looks like you are asking about multiple topics. We will solve one topic at time to avoid confusion. We start with the first topic: {matching_words[0]}.')

            #     elif len(matching_words) == 0:

            #         print(f"\n{self.name}: I am sorry, I do not have information about that topic. Try to rephrase the question or ask me something else.")
            #         continue
            
            #     self.selected_theme = matching_words[0]
            if self.selected_theme is None:
                self.selected_theme = self.themes[np.argmax(cosine_similarity(embed_sentence(query).reshape(1, -1), self.theme_embeddings))]
                print(f'\nConversation Theme: {self.selected_theme}\n')
            answer = self.find_best_answer(query, self.selected_theme)
            print(f"\n{self.name}: {answer}")

        
    def tokenize_sentence(self, sentence):
        
        doc = self.nlp(sentence)
        print(doc)
        tokens = [token.text.lower() for token in doc if not token.is_punct]
        return tokens


    def embed_sentence(self, sentence):
        # Split the sentence into tokens
        tokens = tokenize_sentence(sentence)
        print(tokens)
        # Initialize an empty array to store the word embeddings
        embeddings = np.zeros(self.w2v.vector_size)
        
        n = 0
        # Iterate over each token in the sentence
        for token in tokens:
            # Check if the token is present in the pretrained word2vec model
            if token in self.w2v:
                # Add the word embedding to the sentence embeddings
                embeddings += self.w2v[token]
                n+=1
        
        # Normalize the sentence embeddings
        embeddings /= n+1
        
        return embeddings


    def find_matching_words(self, query):
        return [word for word in self.themes if word in query]
    
    def find_best_answer(self, query, topic):
        
        embed_query = embed_sentence(query).reshape(1, -1)
        if self.current_embedding is None:
            self.current_embedding = embed_query
        
        #df = pd.read_csv(f"word2vec_data/{topic}.csv")
        df_embeddings = np.load(f"word2vec_data/{topic}_embeddings.npy")

        most_similar_responce = calculate_similarity_indices(embed_query , df_embeddings)

        predicted_sentence = self.dataframes[self.selected_theme].iloc[most_similar_responce[0]]['answer']
        self.current_embedding = embed_sentence(predicted_sentence).reshape(1, -1) + embed_query + 0.3 * self.current_embedding
        #print('Closest Words to Embedding:\n')
        #print(self.w2v.similar_by_vector(self.current_embedding.flatten(), topn=5))

        return predicted_sentence
        

In [42]:
# I would like to book a table for 2 people in a restaurant.
c = ChatBot(nlp, w2v)

In [43]:
c.get_dialogue()

MarioBot: Hi, my name is MarioBot, I am a chatbot. I am here to help you with any questions you may have regarding: hotel, train, taxi, bus, hospital, restaurant, attraction.
Please ask me a question:

User: hello hello hello place to get drinks

Conversation Theme: taxi

[[0.3600318  0.42872538 0.38493297 ... 0.40376772 0.26648418 0.4159486 ]]

MarioBot: When would you like the taxi to pick you up?


In [42]:
c.find_matching_words('restaurants attractions')

['restaurant', 'attraction']

In [41]:
c.themes

['hotel', 'train', 'taxi', 'bus', 'hospital', 'restaurant', 'attraction']

In [95]:
df = pd.read_csv('word2vec_data/attraction.csv')
f = df['question_embeddings'].apply(lambda x: np.array(x.split(), dtype=float))

(10953,)

In [75]:
c.find_best_answer('I need a restaurant', 'restaurant')

ValueError: could not convert string to float: '[-5.17966531e-02'

In [101]:
a = np.load('word2vec_data/hospital_embeddings.npy')

In [103]:
a.shape

(219, 300)

In [ ]:
embedding = w2v['Hello']

In [15]:
closest_words = w2v.similar_by_vector(embedding, topn=1)  # Returns top 1 match

In [14]:
closest_words

[('ciao', 1.0000001192092896)]